In [ ]:
import keras
import numpy as np
from keras import layers


: 

In [ ]:
directory=r"D:\Hawk_Eye_dataset-20240209T072717Z-001\Hawk_Eye_dataset\Train"

image_size = (180, 180)
batch_size = 128
train_ds= keras.utils.image_dataset_from_directory(directory,validation_split=None,subset=None,seed=1337,image_size=image_size,batch_size=batch_size,shuffle=True)
train_ds

In [ ]:
data_augmentation_layers = [    layers.RandomFlip("horizontal"),    layers.RandomRotation(0.1)]
def data_augmentation(images):
  for layer in data_augmentation_layers:
    images=layer(images)
  return images

In [ ]:
train_ds=train_ds.map( lambda img, label: (data_augmentation(img), label))

In [ ]:
print(train_ds)

In [ ]:
def make_model(input_shape, num_classes):
  inputs = keras.Input(shape=input_shape)
  # Entry block
  x = layers.Rescaling(1.0 / 255)(inputs)
  x = layers.Conv2D(128, 3, strides=2, padding="same")(x)
  x = layers.BatchNormalization()(x)
  x = layers.Activation("relu")(x)
  previous_block_activation = x # Set aside residual
  for size in [256, 512, 728]:
    x = layers.Activation("relu")(x)
    x = layers.SeparableConv2D(size, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.SeparableConv2D(size, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D(3, strides=2, padding="same")(x)
    # Project residual
    residual = layers.Conv2D(size, 1, strides=2, padding="same")(previous_block_activation)
    x = layers.add([x, residual]) # Add back residual
    previous_block_activation = x # Set aside next residual
  x = layers.SeparableConv2D(1024, 3, padding="same")(x)
  x = layers.BatchNormalization()(x)
  x = layers.Activation("relu")(x)
  x = layers.GlobalAveragePooling2D()(x)
  if num_classes == 10:
    units = 1
  else:
    units = num_classes
  x = layers.Dropout(0.25)(x)
  # We specify activation=None so as to return logits
  outputs = layers.Dense(units, activation=None)(x)
  return keras.Model(inputs, outputs)
model = make_model(input_shape=image_size + (3,), num_classes=10)
keras.utils.plot_model(model, show_shapes=True)




validation dataset

In [ ]:
val_directory="/content/drive/MyDrive/Hawk_Eye_dataset/validation"

image_size = (180, 180)
batch_size = 128
test_ds= keras.utils.image_dataset_from_directory(directory,validation_split=None,subset=None,seed=1337,image_size=image_size,batch_size=batch_size)

In [ ]:
test_ds=test_ds.map( lambda img, label: (data_augmentation(img), label))

In [ ]:
epochs = 25
callbacks =None
model.compile(optimizer=keras.optimizers.Adam(3e-4),loss=keras.losses.BinaryCrossentropy(from_logits=True),metrics=[keras.metrics.BinaryAccuracy(name="acc")])
model.fit(train_ds,epochs=epochs,callbacks=callbacks,validation_data=None)